In [11]:
# Import TensorFlow and enable eager execution
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

# Check if a TPU is available
tpu_available = tf.config.experimental.list_logical_devices('TPU')
print("TPU available:", tpu_available)

# Define a simple TensorFlow operation
x = tf.constant([[1.0, 2.0], [3.0, 4.0]])
y = tf.constant([[5.0, 6.0], [7.0, 8.0]])
z = tf.matmul(x, y)

# Print the result
print("Result of matrix multiplication:")
print(z)


TPU available: []
Result of matrix multiplication:
tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


In [12]:
!pip install gdown


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import gdown

# Define the Google Drive file ID
file_id = '18R9RwjJzhmm-6XuU_DxosLRtsrnkNt_Q'

# Define the URL to download the file
url = f'https://drive.google.com/uc?id={file_id}'

# Define the path to save the downloaded file
output_path = '/content/Books_rating.csv'

# Download the file
gdown.download(url, output_path, quiet=False)

# Load the CSV file into a Pandas DataFrame
df_pandas = pd.read_csv(output_path)

# Display the first few rows and columns of the DataFrame
print("First few rows of the DataFrame:")
print(df_pandas.head())

print("Columns in the DataFrame:")
print(df_pandas.columns)


# Ensure the "review/score" column is of float type
df_pandas["review/score"] = df_pandas["review/score"].astype(float)

# Encode the 'User_id' and 'Id' columns to create 'indexed_user_id' and 'indexed_book_id'
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

df_pandas['indexed_user_id'] = user_encoder.fit_transform(df_pandas['User_id'])
df_pandas['indexed_book_id'] = book_encoder.fit_transform(df_pandas['Id'])

# Define feature columns and target column
feature_columns = ['indexed_user_id', 'indexed_book_id']
target_column = 'review/score'

# Convert to TensorFlow dataset
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_column)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe[feature_columns]), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

# Split the data
train, test = train_test_split(df_pandas, test_size=0.2, random_state=42)

# Create TensorFlow datasets
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# Define the neural network model
from tensorflow.keras import layers

# Define input layers
user_input = layers.Input(shape=(1,), name='user')
book_input = layers.Input(shape=(1,), name='book')

# Embedding layers
user_embedding = layers.Embedding(input_dim=df_pandas['indexed_user_id'].max() + 1, output_dim=50)(user_input)
book_embedding = layers.Embedding(input_dim=df_pandas['indexed_book_id'].max() + 1, output_dim=50)(book_input)

# Concatenate embeddings
concat = layers.concatenate([user_embedding, book_embedding], axis=-1)
concat = layers.Flatten()(concat)

# Fully connected layers
dense = layers.Dense(128, activation='relu')(concat)
dense = layers.Dense(64, activation='relu')(dense)
output = layers.Dense(1)(dense)

# Define the model
model = tf.keras.Model(inputs=[user_input, book_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Prepare the training and validation data
train_data = {'user': train['indexed_user_id'], 'book': train['indexed_book_id']}
train_labels = train['review/score']
val_data = {'user': test['indexed_user_id'], 'book': test['indexed_book_id']}
val_labels = test['review/score']

# Train the model
history = model.fit(
    train_data,
    train_labels,
    epochs=3,
    validation_data=(val_data, val_labels),
    batch_size=batch_size
)

# Evaluate the model
loss, rmse = model.evaluate(val_data, val_labels)
print(f'Root Mean Squared Error (RMSE): {rmse}')

all_user_book_pairs = pd.DataFrame([(user, book) for user in df_pandas['indexed_user_id'].unique() for book in df_pandas['indexed_book_id'].unique()], columns=['indexed_user_id', 'indexed_book_id'])
all_user_book_pairs['predicted_score'] = model.predict({'user': all_user_book_pairs['indexed_user_id'], 'book': all_user_book_pairs['indexed_book_id']})

# Get top N recommendations for each user
N = 5
top_n_recommendations = all_user_book_pairs.groupby('indexed_user_id').apply(lambda x: x.nlargest(N, 'predicted_score')).reset_index(drop=True)

# Create lookup dictionaries for book_id to title and user_id_index to user_id
book_id_to_title = dict(df_pandas[['indexed_book_id', 'Title']].values)
user_index_to_id = dict(df_pandas[['indexed_user_id', 'User_id']].values)

# Print recommendations for each user
for user_id in top_n_recommendations['indexed_user_id'].unique():
    user_recs = top_n_recommendations[top_n_recommendations['indexed_user_id'] == user_id]
    print(f"User: {user_index_to_id[user_id]}")
    print("Top N Recommended Books:")
    for i, row in user_recs.iterrows():
        book_id = row['indexed_book_id']
        title = book_id_to_title[book_id]
        print(f"{i+1}. {title} (Predicted Score: {row['predicted_score']})")
    print("--------")


Downloading...
From (original): https://drive.google.com/uc?id=18R9RwjJzhmm-6XuU_DxosLRtsrnkNt_Q
From (redirected): https://drive.google.com/uc?id=18R9RwjJzhmm-6XuU_DxosLRtsrnkNt_Q&confirm=t&uuid=25f8389f-78b0-43f1-966e-714ba04d6557
To: /content/Books_rating.csv
100%|██████████| 2.86G/2.86G [00:54<00:00, 52.8MB/s]


First few rows of the DataFrame:
           Id                           Title  Price         User_id  \
0  1882931173  Its Only Art If Its Well Hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon    NaN  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon    NaN  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800        

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Generate recommendations
